In [ ]:
#!/usr/local/bin/python3

import hashlib
import os
import binascii


# s is 16 bits long, 32 bit output
def PRG(s: bytes) -> bytes:
    assert len(s) == 2, "You're trying to cheat! Go to Crypto Prison!"
    s_int = int.from_bytes(s, byteorder="big")

    h = hashlib.new("sha3_256")
    h.update(s)

    out = h.digest()

    return out[:4]


def xor_bytes(bytes1: bytes, bytes2: bytes) -> bytes:
    if len(bytes1) != len(bytes2):
        raise ValueError("Byte objects must be of the same length")

    return bytes(b1 ^ b2 for b1, b2 in zip(bytes1, bytes2))


print("In Crypto Civilization, nobody commits the beef. 0 = chicken and 1 = beef.")
print(
    "Every day, Crypto Noobs have 1.5 seconds to do the commitment challenge. Exceptional performers will level up to Crypto Pro."
)
print("Can you level up to a Crypto Pro?")

number_correct = 0

for i in range(200):
    print(f"Crypto test #{i + 1}")

    # generate 32-bit value
    y = os.urandom(4)
    print(f"Here's y: {y.hex()}")

    print("What's your commitment (hex)?")

    com_hex = input("> ").encode()

    try:
        com = binascii.unhexlify(com_hex)
    except:
        print("You're trying to cheat! Go to Crypto Prison!")
        exit(0)

    assert len(com) == 4, "You're trying to cheat! Go to Crypto Prison!"

    choice = int.from_bytes(os.urandom(1), byteorder="big") & 1  # random bit

    if choice == 0:
        print("Did you commit the chicken? Show me (hex).")

    elif choice == 1:
        print("Did you commit the beef? Show me (hex).")

    decom_hex = input("> ").encode()

    try:
        decom = binascii.unhexlify(decom_hex)
    except:
        print("You're trying to cheat! Go to Crypto Prison!")
        exit(0)

    assert len(decom) == 2, "You're trying to cheat! Go to Crypto Prison!"

    if choice == 0:
        if PRG(decom) == com:
            print("Good work. See you tomorrow.")
            number_correct += 1
        else:
            print("Ouch. You don't want to keep doing that.")

    elif choice == 1:
        if xor_bytes(PRG(decom), y) == com:
            print("Good work. See you tomorrow.")
            number_correct += 1
        else:
            print("Ouch. You don't want to keep doing that.")

print(f"{number_correct}/200 trials passed")

if number_correct > 132:
    print("Congrats! You leveled up to a Crypto Pro! Here's your flag.")
    print(open("flag.txt", "r").read())
else:
    print("You messed up too many tasks. You go to Crypto Prison.")


In [57]:
import hashlib
import binascii
import os

# 伪随机数生成器
def PRG(s: bytes) -> bytes:
    assert len(s) == 2, "You're trying to cheat! Go to Crypto Prison!"
    h = hashlib.new("sha3_256")
    h.update(s)
    out = h.digest()
    return out[:4]

# 字节异或函数
def xor_bytes(bytes1: bytes, bytes2: bytes) -> bytes:
    if len(bytes1) != len(bytes2):
        raise ValueError("Byte objects must be of the same length")
    return bytes(b1 ^ b2 for b1, b2 in zip(bytes1, bytes2))

# 给定的 y 值
y_hex = "6210337e"
y = binascii.unhexlify(y_hex)
print(f"使用的 y 值 (十六进制): {y_hex}")

# 随机生成一个长度为 2 字节的 decom
decom = os.urandom(2)

# 情况 1: choice = 0（鸡肉）
com_chicken = PRG(decom)

# 情况 2: choice = 1（牛肉）
com_beef = xor_bytes(PRG(decom), y)

print("情况 1 (choice = 0, 鸡肉):")
print(f"  decom: {decom.hex()}")
print(f"  com: {com_chicken.hex()}")

print("情况 2 (choice = 1, 牛肉):")
print(f"  decom: {decom.hex()}")
print(f"  com: {com_beef.hex()}")

使用的 y 值 (十六进制): 6210337e
情况 1 (choice = 0, 鸡肉):
  decom: 4e78
  com: 88d10191
情况 2 (choice = 1, 牛肉):
  decom: 4e78
  com: eac132ef


In [ ]:
import hashlib
import binascii
import os
import socket

# 伪随机数生成器
def PRG(s: bytes) -> bytes:
    assert len(s) == 2, "You're trying to cheat! Go to Crypto Prison!"
    h = hashlib.new("sha3_256")
    h.update(s)
    out = h.digest()
    return out[:4]

# 字节异或函数
def xor_bytes(bytes1: bytes, bytes2: bytes) -> bytes:
    if len(bytes1) != len(bytes2):
        raise ValueError("Byte objects must be of the same length")
    return bytes(b1 ^ b2 for b1, b2 in zip(bytes1, bytes2))

# 连接到服务器
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect(('chall.lac.tf', 31173))
s.settimeout(5)  # 设置超时时间为 5 秒

try:
    # 接收初始提示信息
    buffer = b""
    while True:
        try:
            data = s.recv(4096)
            if not data:
                break
            buffer += data
            if b"What's your commitment (hex)?" in buffer:
                break
        except socket.timeout:
            break
    print(buffer.decode())

    for i in range(200):
        # 接收测试提示信息
        buffer = b""
        while True:
            try:
                data = s.recv(4096)
                if not data:
                    break
                buffer += data
                if b"Here's y: " in buffer:
                    break
            except socket.timeout:
                break
        print(buffer.decode())

        # 接收 y 的值
        y_hex = buffer.decode().split("Here's y: ")[1].split("\n")[0]
        y = binascii.unhexlify(y_hex)

        # 随机生成一个长度为 2 字节的 decom
        decom = os.urandom(2)

        # 计算 chicken 情况的承诺值
        com_chicken = PRG(decom)

        # 计算 beef 情况的承诺值
        com_beef = xor_bytes(PRG(decom), y)

        # 选择一个承诺值提交，这里选择 com_chicken
        com = com_chicken
        com_hex = com.hex()
        s.send((com_hex + '\n').encode())

        # 接收服务器选择的情况
        buffer = b""
        while True:
            try:
                data = s.recv(4096)
                if not data:
                    break
                buffer += data
                if b"Did you commit the chicken?" in buffer or b"Did you commit the beef?" in buffer:
                    break
            except socket.timeout:
                break
        print(buffer.decode())

        if "beef" in buffer.decode():
            # 如果服务器选择 beef，但我们之前提交的是 com_chicken，需要重新找一个符合 beef 规则的 decom
            found = False
            for j in range(2**16):
                new_decom = j.to_bytes(2, byteorder='big')
                new_com_beef = xor_bytes(PRG(new_decom), y)
                if new_com_beef == com:
                    decom = new_decom
                    found = True
                    break
            if not found:
                print("未找到符合条件的 decom。")
        # 发送揭示值
        decom_hex = decom.hex()
        s.send((decom_hex + '\n').encode())

        # 接收结果信息
        buffer = b""
        while True:
            try:
                data = s.recv(4096)
                if not data:
                    break
                buffer += data
                if b"Good work" in buffer or b"Ouch" in buffer:
                    break
            except socket.timeout:
                break
        print(buffer.decode())

except socket.error as e:
    print(f"Socket 错误: {e}")
finally:
    s.close()

In [2]:
#!/usr/bin/env python3

from pwn import *
import binascii
import hashlib
from tqdm import tqdm


def PRG(s: bytes) -> bytes:
    assert len(s) == 2
    h = hashlib.new("sha3_256")
    h.update(s)

    out = h.digest()
    return out[:4]


def xor_bytes(bytes1: bytes, bytes2: bytes) -> bytes:
    if len(bytes1) != len(bytes2):
        raise ValueError("Byte objects must be of the same length")

    return bytes(b1 ^ b2 for b1, b2 in zip(bytes1, bytes2))


def conn():
    if args.LOCAL:
        r = process(["python3", "chall.py"])

    else:
        r = remote("chall.lac.tf", 31173)

    return r


def main():
    r = conn()

    r.recvline()
    r.recvline()
    r.recvline()

    for i in tqdm(range(200)):
        print(i)
        r.recvline().decode()
        y = binascii.unhexlify(r.recvline().decode().strip().split()[-1].encode())

        doable = False
        send = b""
        seed1 = b""
        seed2 = b""

        mitm = {}

        for i in range(2**16):
            x = PRG(i.to_bytes(2, "big"))
            x2 = xor_bytes(x, y)
            mitm[x2] = i

        for i in range(2**16):
            x2 = PRG(i.to_bytes(2, "big"))
            if x2 in mitm:
                doable = True
                send = PRG(mitm[x2].to_bytes(2, "big"))
                seed1 = mitm[x2].to_bytes(2, "big")
                seed2 = i.to_bytes(2, "big")

        if not doable:
            seed1 = b"\x00a"
            seed2 = b"\x00a"
            send = PRG(b"\x00a")

        r.recvuntil(b"> ")

        r.sendline(send.hex().encode())

        choice = 0 if r.recvline().decode().strip().split()[4] == "chicken?" else 1

        r.recvuntil(b"> ")

        if choice == 0:
            r.sendline(seed1.hex().encode())

        else:
            if doable:
                r.sendline(seed2.hex().encode())
            else:
                r.sendline(b"\x00a".hex().encode())

        print(r.recvline().decode())

    r.interactive()


if __name__ == "__main__":
    main()

[x] Opening connection to chall.lac.tf on port 31173
[x] Opening connection to chall.lac.tf on port 31173: Trying 34.82.120.66
[+] Opening connection to chall.lac.tf on port 31173: Done


  0%|          | 0/201 [00:00<?, ?it/s]

0
Good work. See you tomorrow.



  0%|          | 1/201 [00:01<03:21,  1.01s/it]

1
Good work. See you tomorrow.



...

 99%|█████████▉| 199/201 [03:27<00:02,  1.04s/it]

199
Good work. See you tomorrow.



100%|█████████▉| 200/201 [03:28<00:01,  1.04s/it]

200


Error: Odd-length string